# GENETIC ALGORITHM FOR BIN PACKING

## Imports and data injestion

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [21]:
items = 'items'
bins = 'bins'
items_2D = 'items2D'
bins2D = 'bins2D'

def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns

### Item data

In [19]:
#ONE DIMENSIONAL
df_items1 = read_data('%s.csv'%items)
df_items1.set_index('item')
#TWO DIMENSIONAL
df_items2 = read_data('%s.csv'%items_2D)
df_items2.set_index('item')
display(df_items1)
display(df_items2)

,item,size,bin
0,0,4,NaN
1,1,3,NaN
2,2,6,NaN
3,3,3,NaN
4,4,2,NaN
5,5,1,NaN
6,6,2,NaN


,item,x_size,y_size,bin
0,0,4,2,NaN
1,1,3,1,NaN
2,2,6,8,NaN
3,3,3,2,NaN
4,4,2,1,NaN
5,5,1,1,NaN
6,6,2,2,NaN


# Group Genetic Algorithm (GGA) for Bin Packing Problem (BPP)

##### From: Paper[GGA_FALKENEUR]
Group Genetic Algorithm (GGA), is a modification to the genetic algorithm to suit the structure of grouping problems. We aim to find a good partition of a set, or to group together the elements of the set. 
4 Steps:
1. ENCODING: obtain initial population for each bin, by randomising the order of the items and applying the first ft algorithm. All the solutions are parents.
2. CROSSOVER: Has 5 sub-steps: 
    1. Select the same random crossing site in two parents.
    2. Injest the contents of the first parent at the cross section in at the second parent's cross section, creating children.
    3. Replicated elements are removed.  
    4. Adapt results such that constraints are adhered to. 
    5. Rank the fitness of the children.
3. MUTATION: Shuffle a small group of items among groups.
4. INVERSION: do a random bit swop.

NOTE: IT IS AN UNEDUATED SEARCH SPACE CREATED

## STEP 1: ENCODING
1. Generating the initial population for th $k$ bins. For $n$ items, generate $n$ parents. 
2. Apply first-fit algorithm to each of the $n$ lists. 
3. For each list, we generate a solution for each of the $k$ bins.
4. TABLE: $x$ = LIST$_i$ Bins, and $y$ = Items, filled with bin j accordingly. Where $i \epsilon n$, and $j \epsilon k$  

### First-fit algorithm for 1 and 2 dimensional BPP

In [3]:
### ONE DIMENSIONAL FIRST_FIT ALGORITHM##
def first_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.iat[i,1]
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.iat[j,2]
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,2] = df_vehicles.iat[j,2] - item_size #update avialable space
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,2] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,2] = np.nan
    return df_items, df_vehicles

### TWO DIMENSIONAL FIRST_FIT ALGORITHM##
def first_fit_2D(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size_x = df_items.iat[i,1]
        item_size_y = df_items.iat[i,2]
        while j<= number_of_bins and item_allocated == False:
            available_bin_space_x = df_vehicles.iat[j,3]
            available_bin_space_y = df_vehicles.iat[j,4]
            if available_bin_space_x >= item_size_x and available_bin_space_y >= item_size_y: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,3] = df_vehicles.iat[j,3] - item_size_x #update avialable space in x direction
                df_vehicles.iat[j,4] = df_vehicles.iat[j,4] - item_size_y #update avialable space in y direction 
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,3] = bin_num#set the allocated bin for the item
            elif available_bin_space_x >= item_size_y and available_bin_space_y >= item_size_x: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.iat[j,3] = df_vehicles.iat[j,3] - item_size_y #update avialable space in x direction
                df_vehicles.iat[j,4] = df_vehicles.iat[j,4] - item_size_x #update avialable space in y direction 
                bin_num = df_vehicles.iat[j,0]
                df_items.iat[i,3] = bin_num#set the allocated bin for the item
            elif available_bin_space_x < item_size_x or available_bin_space_y < item_size_y: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.iat[i,3] = np.nan
    return df_items, df_vehicles

In [ ]:
def endoding(df_items, df_vehicles, dimesion):
    df_items_copy = df_items.copy()
    n = df_items.shape[0] #number of items
    k = df_vehicles.shape[0]#number of bins
    newDF = pd.DataFrame() #initialise a new dataframe
    if dimension == 1:
        newDF = newDF.append(df_items[:][['item','size']])
        #use 1d first fit
        for i in range n: 
            df_items_copy_temp = shuffle(df_items_copy) #shuffle the copy of items
            df_items_copy_temp = first_fit(df_items_copy_temp) #assigns items to bins using ff
            df_items_copy_temp = df_items_copy_temp.sort_values(by = 'item').reset_index(drop=True) #rearranges the list according to item 0,1,2... and resets the index as well
    elif dimesion == 2:
        newDF = newDF.append(df_items[:][['item','x_size','y_size']])
        #use 2d first fit
        for i in range n: 
            df_items_copy_temp = shuffle(df_items_copy) #shuffle the copy of items
            df_items_copy_temp = first_fit_2D(df_items_copy_temp) #assigns items to bins using ff 
